In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/its-a-fraud/train.csv
/kaggle/input/its-a-fraud/test.csv
/kaggle/input/its-a-fraud/mock_submission.csv


In [4]:
dfTrain = pd.read_csv('../input/train-test-v2/preprocessesTrainv2.csv')
dfTest = pd.read_csv('../input/train-test-v2/preprocessedTestV2.csv')

In [20]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(dfTrain,random_state=104, test_size=0.25, shuffle=True)

In [10]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

In [11]:
XGBClassifier()

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)

In [23]:
def XGBWithStratifiedKFold(df):
    
    X = df.drop(['isFraud'],axis=1)
    Y = df['isFraud']
    
    
    X = X-X.min()/X.max()-X.min()
    
    skf = StratifiedKFold(n_splits=5,random_state=9,shuffle=True)
    model = XGBClassifier(n_estimators=500,max_depth=15,n_jobs=-1)
    
    
    
    for train_index,test_index in skf.split(X,Y):
        
        X_train = X.iloc[train_index]
        Y_train = Y.iloc[train_index]
        
        X_test = X.iloc[test_index]
        Y_test = Y.iloc[test_index]
        
        model.fit(X_train,Y_train)
        
        
        print("Accuracy ", model.score(X_test,Y_test))
        
        prob = model.predict_proba(X_test)[:,1]
        
        auc = metrics.roc_auc_score(Y_test,prob)
        print("AUC : ",auc,"\n===============================")
    
    return model

In [26]:
model = XGBWithStratifiedKFold(x)

Accuracy  0.9858120255047271
AUC :  0.9574143586263448 
Accuracy  0.9861741615902948
AUC :  0.9589494989421165 
Accuracy  0.9855404234405515
AUC :  0.9583704730595377 
Accuracy  0.9853591660415427
AUC :  0.9542078377953511 
Accuracy  0.9856825060141234
AUC :  0.9559582743234468 


In [27]:
def evaluateModel(df,model):
    
    y_pred = model.predict(df)
    
    result = pd.DataFrame(y_pred)
    result.columns = ['isFraud']
    result.to_csv("./result.csv")
    
    print("Result saved file")    

In [28]:
evaluateModel(dfTest,model)

Result saved file


# **Searching Parameters**

In [25]:
import sklearn
hyperparameter_grid1 = {
    'n_estimators': [50,100, 150,200,250,300,350,400,450,500],
    'max_depth': [ 9, 12 , 16, 20 , 25],
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.2, 0.25, 0.3],
    'min_child_weight' :[1,5,100],
    'colsample_bylevel':[0.1,0.2,0.4,0.8,1],
    'colsample_bytree' : [0.1,0.2,0.4,0.8,1]
}
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=4,random_state=1,shuffle=True)
random_cv_skf = sklearn.model_selection.RandomizedSearchCV(estimator=XGBClassifier(),
            param_distributions=hyperparameter_grid1,
            cv=kf, 
            n_iter=5,
            scoring = 'roc_auc',
            n_jobs = -1,
            verbose = 5, 
            return_train_score = True,
            random_state=42)



random_cv_skf.fit(train.drop(['isfraud'],axis=1), train['isFraud'])
model = random_cv_skf.best_estimator_
random_cv_skf.best_params_

NameError: name 'sklearn' is not defined